#### considering the cutoff date before a promo is shown is 10-09-2011.

In [23]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\vinee\OneDrive\Desktop\projects\A-B Testing\data\cleaned_data.csv")

In [24]:
df = df.drop(columns=["Unnamed: 0"])

In [25]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

In [27]:
df.shape

(397924, 10)

In [33]:
df.dtypes

InvoiceNo                   int64
StockCode                  object
Description                object
Quantity                    int64
InvoiceDate        datetime64[ns]
UnitPrice                 float64
CustomerID                  int64
Country                    object
InvoiceDateOnly            object
TotalPrice                float64
dtype: object

In [29]:
cutoff = pd.to_datetime("2011-10-09")

In [30]:
df_pre = df[df["InvoiceDate"] < cutoff]

In [36]:
df_pre["InvoiceDate"].dtype

dtype('<M8[ns]')

In [37]:
# Customer behaviour features

customer = df_pre.groupby("CustomerID").agg(
    recency_days = ("InvoiceDate", lambda x: (cutoff - x.max()).days),
    frequency = ("InvoiceNo", "nunique"),
    monetary_funds = ("TotalPrice", "sum"),
    avg_order_value = ("TotalPrice", lambda x: x.sum() / x.count()),
    avg_basket_size = ("Quantity", "mean"),
    total_quantity = ("Quantity", "sum"),
    unique_products = ("StockCode", "nunique"),
    unique_invoices = ("InvoiceNo", "nunique"),
    tenure_days = ("InvoiceDate", lambda x: (x.max() - x.min()).days),
    most_common_hour = ("InvoiceDate", lambda x: x.dt.hour.mode()[0]),
    most_common_weekday = ("InvoiceDate", lambda x: x.dt.weekday.mode()[0]),
    last_purchase_month = ("InvoiceDate", lambda x: x.max().month)
).reset_index()

In [38]:
customer.head()

,CustomerID,recency_days,frequency,monetary_funds,avg_order_value,avg_basket_size,total_quantity,unique_products,unique_invoices,tenure_days,most_common_hour,most_common_weekday,last_purchase_month
0,12346,263,1,77183.60,77183.600000,74215.000000,74215,1,1,0,10,1,1
1,12347,67,5,2790.86,22.506935,12.822581,1590,82,5,237,14,1,8
2,12348,13,4,1797.24,57.975484,75.516129,2341,22,4,282,19,3,9
3,12350,248,1,334.40,19.670588,11.588235,197,17,1,0,16,2,2
4,12352,10,7,2194.31,31.347286,5.842857,409,47,7,224,14,1,9


In [39]:
customer.to_csv(r"C:\Users\vinee\OneDrive\Desktop\projects\A-B Testing\data\Customer.csv")